In [1]:

import tensorflow as tf
import chess
import chess_functions as cfn
import nn_functions as nnfn


2023-11-20 16:20:52.661266: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
nnfn.test_gpu()


model = nnfn.define_model()
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 8, 8, 16)          224       
                                                                 
 block1_conv2 (Conv2D)       (None, 8, 8, 32)          544       
                                                                 
 block1_conv3 (Conv2D)       (None, 8, 8, 32)          1056      
                                                                 
 resizing (Resizing)         (None, 128, 128, 32)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 64)      2112      
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     8320      
                                                                 
 block2_conv3 (Conv2D)       (None, 128, 128, 128)     